In [1]:
import pandas as pd
import numpy as np
import os
from astropy.cosmology import Planck13
from astropy.io import fits
from astropy.table import Table
from tqdm import tqdm
%matplotlib inline
import matplotlib.pyplot as plt

import disperse

from sklearn.neighbors import KDTree

from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold
import lightgbm as lgbm

In [2]:
cosmo = Planck13
H0 = cosmo.H0.value
Om = cosmo.Om0
Ol = 0.69288
Ok = 0.0
print(f'H0 = {cosmo.H0.value}')
print(f'Om = {cosmo.Om0}')
print(f'Ol = {0.69288}')

H0 = 67.77
Om = 0.30712
Ol = 0.69288


In [3]:
np.random.seed(0)

In [4]:
gal_RA_int = (140, 260)
gal_DEC_int = (-10, 30)
gal_Z_int = (0, 1.0)

In [5]:
galaxies = pd.read_csv('SDSS/SDSS_DR16.csv')
galaxies = galaxies[galaxies['class'] == 'GALAXY']
galaxies = galaxies[(gal_RA_int[0] <= galaxies['ra']) & (galaxies['ra'] <= gal_RA_int[1])]
galaxies = galaxies[(gal_DEC_int[0] <= galaxies['dec']) & (galaxies['dec'] <= gal_DEC_int[1])]
galaxies = galaxies[(gal_Z_int[0] < galaxies['z']) & (galaxies['z'] <= gal_Z_int[1])]
galaxies = galaxies[['ra', 'dec', 'z']]
galaxies.columns = ['RA', 'DEC', 'Z']
galaxies.drop_duplicates(subset=['RA', 'DEC', 'Z'])
galaxies.reset_index(drop=True, inplace=True)

In [6]:
util_DPS = disperse.Disperse3D(
    galaxies, '_disperse_03/bin/',
    H0, Om, Ol, Ok,
    clusters=None
)

In [7]:
cl_RA_int = (160, 240)
cl_DEC_int = (0, 20)
cl_Z_int = (0, 1.0)

In [8]:
dat = Table.read('DR5_cluster-catalog_v1.1.fits', format='fits')
true_clusters = dat[['RADeg', 'decDeg', 'redshift', 'redshiftType', 'M500c']].to_pandas()
true_clusters = true_clusters[true_clusters['redshiftType'] == b'spec']
true_clusters = true_clusters[['RADeg', 'decDeg', 'redshift', 'M500c']]
true_clusters.columns = ['RA', 'DEC', 'Z', 'M']
true_clusters = true_clusters[(cl_RA_int[0] <= true_clusters['RA']) & (true_clusters['RA'] <= cl_RA_int[1])]
true_clusters = true_clusters[(cl_DEC_int[0] <= true_clusters['DEC']) & (true_clusters['DEC'] <= cl_DEC_int[1])]
true_clusters = true_clusters[(cl_Z_int[0] < true_clusters['Z']) & (true_clusters['Z'] <= cl_Z_int[1])]
true_clusters = true_clusters[['RA', 'DEC', 'Z', 'M']]
true_clusters.columns = ['RA', 'DEC', 'Z', 'M']
true_clusters.drop_duplicates(subset=['RA', 'DEC', 'Z', 'M'])
true_clusters = true_clusters.sort_values(by=['Z'])
true_clusters.reset_index(drop=True, inplace=True)
true_clusters['R'] = [0] * true_clusters.shape[0]
true_clusters['type'] = [1] * true_clusters.shape[0]
CX, CY, CZ = util_DPS.sph2cart(true_clusters['RA'], true_clusters['DEC'], true_clusters['Z'])
true_clusters['CX'] = CX
true_clusters['CY'] = CY
true_clusters['CZ'] = CZ
true_clusters['ID'] = list(range(true_clusters.shape[0]))
true_clusters

,RA,DEC,Z,M,R,type,CX,CY,CZ,ID
0,230.761495,8.587807,0.035200,1.676978,0,1,-96.597031,-118.277283,23.062053,0
1,230.452957,7.709549,0.044200,2.457475,0,1,-122.101803,-147.873793,25.960786,1
2,227.733561,5.744353,0.076600,11.002140,0,1,-222.703294,-245.035914,33.308899,2
3,208.253128,5.139146,0.078800,1.222567,0,1,-300.190348,-161.319215,30.649445,3
4,173.207731,14.465334,0.083200,2.935207,0,1,-346.994931,41.329156,90.147741,4
...,...,...,...,...,...,...,...,...,...,...
514,190.801637,13.213381,0.789772,1.827705,0,1,-2716.094276,-518.203135,649.226460,514
515,160.265704,10.675110,0.789868,4.328044,0,1,-2627.451148,942.538451,526.183483,515
516,199.909131,15.890405,0.796336,1.899555,0,1,-2585.211521,-936.299514,782.729567,516
517,211.772844,10.806421,0.836000,6.430338,0,1,-2479.343209,-1535.631547,556.669132,517


In [9]:
#генерация случайных скоплений
RANDOM_CL_NUM = 3000
TRUE_R = 20
TRUE_CL_NUM = true_clusters.shape[0]

RA_int = (true_clusters['RA'].min(), true_clusters['RA'].max())
DEC_int = (true_clusters['DEC'].min(), true_clusters['DEC'].max())
Z_int = (true_clusters['Z'].min(), np.quantile(true_clusters['Z'], [1])[0])

CX, CY, CZ = util_DPS.sph2cart(
    true_clusters['RA'], true_clusters['DEC'], true_clusters['Z']
)
points = [[CX[i], CY[i], CZ[i]] for i in range(true_clusters.shape[0])]
kd_tree = KDTree(points, leaf_size=2)

not_generate = RANDOM_CL_NUM
RA, DEC, Z = np.array([]), np.array([]), np.array([])
while not_generate > 0:
    print('###')
    RA_ = np.random.uniform(RA_int[0], RA_int[1], not_generate)
    DEC_ = np.random.uniform(DEC_int[0], DEC_int[1], not_generate)
    
#     Z_ = np.random.uniform(Z_int[0], Z_int[1], not_generate)
    noise = np.random.normal(0, 0.005, not_generate)
    Z_ = np.random.choice(true_clusters['Z'], not_generate) + noise
    
    CX, CY, CZ = util_DPS.sph2cart(RA_, DEC_, Z_)
    points = [[CX[i], CY[i], CZ[i]] for i in range(not_generate)]
    inters = kd_tree.query_radius(points, TRUE_R, count_only=True)
    RA = np.concatenate((RA, RA_[inters==0]))
    DEC = np.concatenate((DEC, DEC_[inters==0]))
    Z = np.concatenate((Z, Z_[inters==0]))
    not_generate = RANDOM_CL_NUM - len(RA)
    
M = [0] * RANDOM_CL_NUM
R = [0] * RANDOM_CL_NUM
type_ = [0] * RANDOM_CL_NUM

false_clusters = pd.DataFrame({
    'RA': RA, 'DEC': DEC, 'Z': Z, 'M': M, 'R': R, 'type': type_
})
false_clusters = false_clusters.sort_values(by=['Z'])
false_clusters.reset_index(drop=True, inplace=True)
CX, CY, CZ = util_DPS.sph2cart(false_clusters['RA'], false_clusters['DEC'], false_clusters['Z'])
false_clusters['CX'] = CX
false_clusters['CY'] = CY
false_clusters['CZ'] = CZ
false_clusters['ID'] = true_clusters.shape[0] + np.array(range(false_clusters.shape[0]))
false_clusters

###
###


,RA,DEC,Z,M,R,type,CX,CY,CZ,ID
0,195.167266,7.873423,0.027191,0,0,0,-114.274601,-30.977626,16.373231,519
1,200.850116,0.652633,0.027575,0,0,0,-113.261390,-43.137368,1.380577,520
2,235.364127,6.082430,0.029708,0,0,0,-73.760981,-106.779534,13.829108,521
3,201.651001,2.338991,0.031765,0,0,0,-129.533638,-51.419418,5.692519,522
4,206.929088,0.621350,0.032173,0,0,0,-125.937647,-63.972227,1.531906,523
...,...,...,...,...,...,...,...,...,...,...
2995,167.092779,2.064009,0.969065,0,0,0,-3234.437622,741.214118,119.588654,3514
2996,172.879468,1.072602,0.970024,0,0,0,-3296.663668,411.820925,62.201945,3515
2997,215.591571,2.062434,0.970373,0,0,0,-2701.080393,-1933.182671,119.616876,3516
2998,239.195030,11.486234,0.973061,0,0,0,-1671.481723,-2803.383711,663.224103,3517


In [10]:
clusters = pd.concat((true_clusters, false_clusters), axis=0)
clusters.reset_index(drop=True, inplace=True)
clusters

,RA,DEC,Z,M,R,type,CX,CY,CZ,ID
0,230.761495,8.587807,0.035200,1.676978,0,1,-96.597031,-118.277283,23.062053,0
1,230.452957,7.709549,0.044200,2.457475,0,1,-122.101803,-147.873793,25.960786,1
2,227.733561,5.744353,0.076600,11.002140,0,1,-222.703294,-245.035914,33.308899,2
3,208.253128,5.139146,0.078800,1.222567,0,1,-300.190348,-161.319215,30.649445,3
4,173.207731,14.465334,0.083200,2.935207,0,1,-346.994931,41.329156,90.147741,4
...,...,...,...,...,...,...,...,...,...,...
3514,167.092779,2.064009,0.969065,0.000000,0,0,-3234.437622,741.214118,119.588654,3514
3515,172.879468,1.072602,0.970024,0.000000,0,0,-3296.663668,411.820925,62.201945,3515
3516,215.591571,2.062434,0.970373,0.000000,0,0,-2701.080393,-1933.182671,119.616876,3516
3517,239.195030,11.486234,0.973061,0.000000,0,0,-1671.481723,-2803.383711,663.224103,3517


In [11]:
sigmas = [round(i, 1) for i in np.arange(0.2, 14.2, 0.2)]

In [12]:
f_name = 'ACT_05_dists.npy'
if os.path.exists(f_name):
    with open(f_name, 'rb') as f:
        dists = np.load(f)
else:
    dists = []
    for sigma in tqdm(sigmas):
        DPS = disperse.Disperse3D.read(f'ACT_01_dumps/{sigma}/')

        _, _, cl_dists \
            = DPS.count_conn(clusters['R'], clusters)
        dists.append(np.array(cl_dists)[None,:])
    dists = np.concatenate(dists, axis=0)
    dists = dists.T
    with open(f_name, 'wb') as f:
        np.save(f, dists)
    
print(dists.shape)

(3519, 70)


In [13]:
z = [round(i, 4) for i in np.arange(0.002, 1.0, 0.002)]

ID, RA, DEC, Z, Z_true, M, R, TYPE = [], [], [], [], [], [], [], []
for i in range(clusters.shape[0]):
    row = clusters.iloc[i]
    ID = ID + [int(row['ID'])] * len(z)
    RA = RA + [row['RA']] * len(z)
    DEC = DEC + [row['DEC']] * len(z)
    Z = Z + z.copy()
    Z_true = Z_true + [row['Z']] * len(z)
    TYPE = TYPE + [int(row['type'])] * len(z)
    M = M + [row['M']] * len(z)
    R = R + [row['R']] * len(z)
CX, CY, CZ = util_DPS.sph2cart(RA, DEC, Z)
clusters_ext = pd.DataFrame({
    'ID': ID, 'RA': RA, 'DEC': DEC, 'Z': Z,
    'CX': CX, 'CY': CY, 'CZ': CZ,
    'Z_true': Z_true, 'M': M, 'R': R, 'type': TYPE
})

clusters_ext

,ID,RA,DEC,Z,CX,CY,CZ,Z_true,M,R,type
0,0,230.761495,8.587807,0.002,-5.531095,-6.772495,1.320521,0.035200,1.676978,0.0,1
1,0,230.761495,8.587807,0.004,-11.057085,-13.538739,2.639823,0.035200,1.676978,0.0,1
2,0,230.761495,8.587807,0.006,-16.577964,-20.298725,3.957905,0.035200,1.676978,0.0,1
3,0,230.761495,8.587807,0.008,-22.093726,-27.052446,5.274765,0.035200,1.676978,0.0,1
4,0,230.761495,8.587807,0.010,-27.604365,-33.799893,6.590403,0.035200,1.676978,0.0,1
...,...,...,...,...,...,...,...,...,...,...,...
1755976,3518,162.522675,6.962019,0.990,-3193.800208,1005.611924,408.875850,0.978053,0.000000,0.0,0
1755977,3518,162.522675,6.962019,0.992,-3198.544998,1007.105886,409.483287,0.978053,0.000000,0.0,0
1755978,3518,162.522675,6.962019,0.994,-3203.284232,1008.598099,410.090012,0.978053,0.000000,0.0,0
1755979,3518,162.522675,6.962019,0.996,-3208.017918,1010.088566,410.696026,0.978053,0.000000,0.0,0


In [14]:
# f_name = 'ACT_05_dists_ext.npy'
# if os.path.exists(f_name):
#     with open(f_name, 'rb') as f:
#         dists_ext = np.load(f)
# else:
#     dists_ext = []
#     for sigma in tqdm(sigmas):
#         DPS = disperse.Disperse3D.read(f'ACT_01_dumps/{sigma}/')

#         _, _, cl_dists \
#             = DPS.count_conn(clusters_ext['R'], clusters_ext)
#         dists_ext.append(np.array(cl_dists)[None,:])
#     dists_ext = np.concatenate(dists_ext, axis=0)
#     dists_ext = dists_ext.T
#     with open(f_name, 'wb') as f:
#         np.save(f, dists_ext)

dists_ext = np.zeros((clusters_ext.shape[0], len(sigmas)))
print(dists_ext.shape)

(1755981, 70)


In [15]:
TEST_RATIO = 0.1
select_step = int(1.01 // TEST_RATIO)

mask = np.array([True] * clusters.shape[0])
mask[::select_step] = False

train_clusters = clusters.loc[mask]
train_clusters.reset_index(drop=True, inplace=True)
test_clusters = clusters.loc[~mask]
test_clusters.reset_index(drop=True, inplace=True)

In [16]:
train_clusters

,RA,DEC,Z,M,R,type,CX,CY,CZ,ID
0,230.452957,7.709549,0.044200,2.457475,0,1,-122.101803,-147.873793,25.960786,1
1,227.733561,5.744353,0.076600,11.002140,0,1,-222.703294,-245.035914,33.308899,2
2,208.253128,5.139146,0.078800,1.222567,0,1,-300.190348,-161.319215,30.649445,3
3,173.207731,14.465334,0.083200,2.935207,0,1,-346.994931,41.329156,90.147741,4
4,163.466549,16.844133,0.085400,2.420530,0,1,-339.698430,100.839024,107.282298,5
...,...,...,...,...,...,...,...,...,...,...
3162,167.092779,2.064009,0.969065,0.000000,0,0,-3234.437622,741.214118,119.588654,3514
3163,172.879468,1.072602,0.970024,0.000000,0,0,-3296.663668,411.820925,62.201945,3515
3164,215.591571,2.062434,0.970373,0.000000,0,0,-2701.080393,-1933.182671,119.616876,3516
3165,239.195030,11.486234,0.973061,0.000000,0,0,-1671.481723,-2803.383711,663.224103,3517


In [17]:
test_clusters

,RA,DEC,Z,M,R,type,CX,CY,CZ,ID
0,230.761495,8.587807,0.035200,1.676978,0,1,-96.597031,-118.277283,23.062053,0
1,235.775108,5.770824,0.105011,1.503079,0,1,-253.544682,-372.731247,45.557881,10
2,217.958949,13.531505,0.159770,1.672366,0,1,-521.321295,-406.699659,159.123752,20
3,228.275559,6.190439,0.174354,1.404034,0,1,-489.261405,-548.663568,79.735996,30
4,221.662409,17.849673,0.200000,2.159093,0,1,-599.264179,-533.220052,258.308824,40
...,...,...,...,...,...,...,...,...,...,...
347,164.187561,15.561575,0.771632,0.000000,0,0,-2584.883178,732.054353,748.155071,3470
348,192.631368,3.771055,0.778508,0.000000,0,0,-2734.459010,-612.796023,184.705503,3480
349,198.207006,5.906494,0.785264,0.000000,0,0,-2671.709670,-878.775442,290.968013,3490
350,172.017732,13.110517,0.790565,0.000000,0,0,-2741.605761,384.442354,644.769273,3500


In [30]:
train_clusters_ext = clusters_ext[clusters_ext['ID'].isin(train_clusters['ID'])]
train_clusters_ext.reset_index(drop=True, inplace=True)
train_clusters_ext

,ID,RA,DEC,Z,CX,CY,CZ,Z_true,M,R,type
0,1,230.452957,7.709549,0.002,-5.579718,-6.757427,1.186337,0.044200,2.457475,0.0,1
1,1,230.452957,7.709549,0.004,-11.154286,-13.508618,2.371579,0.044200,2.457475,0.0,1
2,1,230.452957,7.709549,0.006,-16.723699,-20.253565,3.555724,0.044200,2.457475,0.0,1
3,1,230.452957,7.709549,0.008,-22.287949,-26.992260,4.738773,0.044200,2.457475,0.0,1
4,1,230.452957,7.709549,0.010,-27.847031,-33.724696,5.920722,0.044200,2.457475,0.0,1
...,...,...,...,...,...,...,...,...,...,...,...
1580328,3518,162.522675,6.962019,0.990,-3193.800208,1005.611924,408.875850,0.978053,0.000000,0.0,0
1580329,3518,162.522675,6.962019,0.992,-3198.544998,1007.105886,409.483287,0.978053,0.000000,0.0,0
1580330,3518,162.522675,6.962019,0.994,-3203.284232,1008.598099,410.090012,0.978053,0.000000,0.0,0
1580331,3518,162.522675,6.962019,0.996,-3208.017918,1010.088566,410.696026,0.978053,0.000000,0.0,0


In [31]:
test_clusters_ext = clusters_ext[clusters_ext['ID'].isin(test_clusters['ID'])]
test_clusters_ext.reset_index(drop=True, inplace=True)
test_clusters_ext

,ID,RA,DEC,Z,CX,CY,CZ,Z_true,M,R,type
0,0,230.761495,8.587807,0.002,-5.531095,-6.772495,1.320521,0.035200,1.676978,0.0,1
1,0,230.761495,8.587807,0.004,-11.057085,-13.538739,2.639823,0.035200,1.676978,0.0,1
2,0,230.761495,8.587807,0.006,-16.577964,-20.298725,3.957905,0.035200,1.676978,0.0,1
3,0,230.761495,8.587807,0.008,-22.093726,-27.052446,5.274765,0.035200,1.676978,0.0,1
4,0,230.761495,8.587807,0.010,-27.604365,-33.799893,6.590403,0.035200,1.676978,0.0,1
...,...,...,...,...,...,...,...,...,...,...,...
175643,3510,200.693154,12.028981,0.990,-3086.335847,-1165.807748,703.006335,0.842273,0.000000,0.0,0
175644,3510,200.693154,12.028981,0.992,-3090.920985,-1167.539701,704.050739,0.842273,0.000000,0.0,0
175645,3510,200.693154,12.028981,0.994,-3095.500754,-1169.269626,705.093920,0.842273,0.000000,0.0,0
175646,3510,200.693154,12.028981,0.996,-3100.075163,-1170.997527,706.135879,0.842273,0.000000,0.0,0


In [18]:
FOLDS_NUM = 2

train_ID, train_Y = train_clusters['ID'], train_clusters['type']

skf = StratifiedKFold(n_splits=FOLDS_NUM)
skf.get_n_splits(train_ID, train_Y)

2

In [19]:
train_feas = np.concatenate(
    (dists[train_clusters['ID'].values], train_clusters['Z'].values.reshape(-1, 1)), axis=1
)
train_feas.shape

(3167, 71)

In [20]:
rads = list(range(1, 31))

In [21]:
sigma_scores = np.zeros((train_clusters.shape[0], len(rads)))
for train_index, test_index in skf.split(train_ID, train_Y):
    t = []
    for i, rad in tqdm(enumerate(rads)):
        s = np.zeros(test_index.shape[0])
        for j, sigma in enumerate(sigmas):
            s[train_feas[test_index, j] <= rad] = sigma
        t.append(s)
    t = np.array(t).T
    sigma_scores[test_index] = t
    
sigma_rocaucs = []
for i in range(len(rads)):
    sigma_rocaucs.append(roc_auc_score(train_Y, sigma_scores[:, i]))

sigma_rocaucs

30it [00:00, 1442.08it/s]
30it [00:00, 1571.94it/s]


[0.609483702117535,
 0.6616349432944723,
 0.7037223411848681,
 0.7242818621619478,
 0.7525580934253311,
 0.7812502974066143,
 0.7950059481322864,
 0.8064081211832818,
 0.8157371718613687,
 0.8150118962645729,
 0.8182345943373781,
 0.8177833293679118,
 0.8190356094852882,
 0.8145078911888334,
 0.8147018003013721,
 0.8124581648029184,
 0.8090542469664526,
 0.8081422000158618,
 0.8014350860496471,
 0.7934483305575383,
 0.7824775953683876,
 0.7757518439210089,
 0.7703354746609565,
 0.7655166944246174,
 0.7590423507018795,
 0.7542588627171067,
 0.7449119676421604,
 0.740232770243477,
 0.7342156396224918,
 0.7248857958601]

In [23]:
logregs = []
logreg_scores = np.zeros(train_clusters.shape[0])
logreg_rocaucs = []
for train_index, test_index in skf.split(train_ID, train_Y):
    logreg = LogisticRegression(
        random_state=0, max_iter=10000, C=0.0005
    ).fit(train_feas[train_index], train_Y[train_index])
    logregs.append(logreg)
    preds = logreg.predict_proba(train_feas[test_index])[:,1].reshape(-1)
    logreg_scores[test_index] = preds
    logreg_rocaucs.append(roc_auc_score(train_Y[test_index], preds))
    
logreg_rocaucs

[0.6690345045900602, 0.7641328882530599]

In [25]:
boostings = []
boosting_scores = np.zeros(train_clusters.shape[0])
boosting_rocaucs = []
for train_index, test_index in skf.split(train_ID, train_Y):
    boosting = lgbm.LGBMClassifier(
        boosting_type='gbdt', objective='binary',
        learning_rate=0.01, n_estimators=200,
        max_depth=10, bagging_fraction=1.0,
        reg_lambda=0.2
    ).fit(train_feas[train_index], train_Y[train_index])
    boostings.append(boosting)
    preds = boosting.predict_proba(train_feas[test_index])[:,1].reshape(-1)
    boosting_scores[test_index] = preds
    boosting_rocaucs.append(roc_auc_score(train_Y[test_index], preds))
    
boosting_rocaucs

[LightGBM] [Warning] bagging_fraction is set=1.0, subsample=1.0 will be ignored. Current value: bagging_fraction=1.0
[LightGBM] [Warning] bagging_fraction is set=1.0, subsample=1.0 will be ignored. Current value: bagging_fraction=1.0


[0.8752548274770496, 0.8077412176124622]

In [26]:
rfs = []
rf_scores = np.zeros(train_clusters.shape[0])
rf_rocaucs = []
for train_index, test_index in skf.split(train_ID, train_Y):
    rf = RandomForestClassifier(
        max_depth=10, 
        random_state=0,
        n_estimators=500,
        max_features=None
    ).fit(train_feas[train_index], train_Y[train_index])
    rfs.append(rf)
    preds = rf.predict_proba(train_feas[test_index])[:,1].reshape(-1)
    rf_scores[test_index] = preds
    rf_rocaucs.append(roc_auc_score(train_Y[test_index], preds))
    
rf_rocaucs

[0.33868471035137704, 0.8279939596248608]

In [ ]:
sigma_scores_ext = np.zeros((.shape[0], len(rads)))
for train_index, test_index in skf.split(train_ID, train_Y):
    
    